# F-тест: линейные ограничения

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from scipy.stats import f # f-распределение и критические значения

# Не показывать FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<i><b><span style="color: purple">Значимость регрессии </span> </b><br>

Для линейной регрессии

$$
	(\log)y=\beta_0+\beta_1(\log)x_1+\cdots+\beta_k(\log)x_k+u=x^\top \beta+u
$$

Тестируется гипотеза, что коэффициенты модели удовлетворяют системе $J$ линейных ограничений (запишем их в матричном виде)

$$
H_0: R\beta=q
$$

с матрицами $\underset{J\times(k+1)}{R}$ и $\underset{J\times(1)}{q}$. Будем предполагать, что ограничения линейно независимы, т.е. $rank (R)=J$

__Интерпретация__: зависит от модели/контекста

__Тестовая статистика__ для выбранной оценки ковариационной матрицы $\hat{V}$ формула для F-статистики (всё вычисляется автоматически!)

$$
	F=\frac{1}{J}(R\widehat{\beta}-q)^\top
	\Bigl(R\cdot\hat{V}\cdot R^\top\Bigr)^{-1}
	(R\widehat{\beta}-q) 
$$

__Критическое значение__: $F$-распределения со степенями свободы ($k$ - число объясняющих переменных)

$$ 
\begin{aligned} 
	df_1&=dfn=J & df_2&=dfd=n-k-1
\end{aligned}
$$

__Замечание__: 
* для F-тест есть неробастный (по умолчанию) и робастный вариант
* для подогнанной регрессии используем метод `.f_test(r_matrix)` или `.wald_test(r_matrix, use_f=True)`
* тестируемую гипотезу специфицируем в виде `'x1=x2=0'` или `'x1=0,x2=0'` (параметр `r_matrix`) 

__Вывод__

**<span style="color:purple">Способ 1:</span>** используем $F$-статистику

* Отвергаем гипотезу $H_0$ при $F>F_{cr}$, <i><b><span style="color: purple">совместное влияния значимо</span></b><br>
* Не отвергаем гипотезу $H_0$ при $F<F_{cr}$, <i><b><span style="color: purple">совместное влияния незначимо</span> </b><br>

**<span style="color:purple">Способ 2:</span>** используем $P-value$

* Отвергаем гипотезу $H_0$  при $P<\alpha$, <i><b><span style="color: purple">совместное влияния значимо</span> </b><br>
* Не отвергаем гипотезу $H_0$ при $P>\alpha$, <i><b><span style="color: purple">совместное влияния незначимо</span> </b><br>

</div>

Для набора данных `Labour` рассмотрим линейную регрессию __log(output)~log(caputal)+log(labour)+log(wage)__

Тестируем гипотезу

$$
	H_0:\beta_{capital}+\beta_{labour}+\beta_{wage}=1
$$

Для этой гипотезы $J=1$

In [2]:
# импорт данных
df = pd.read_csv('Labour.csv')

## F-тест (неробастный)

Используем OLS-оценку ковариационной матрицы $\hat{V}=s^2(X^\top X)^{-1}$

In [3]:
# спецификация модели через формулу
mod = smf.ols(formula='np.log(output)~1+np.log(capital)+np.log(labour)+np.log(wage)', data=df)
# подгонка модели с неробастной оценкой ковариационной матрицы
res_ols = mod.fit()

In [4]:
F_test = res_ols.f_test('np.log(capital)+np.log(labour)+np.log(wage)=1')
print(F_test)

<F test: F=198.58693315849226, p=7.280959454662307e-39, df_denom=565, df_num=1>


In [5]:
# Тестовая статистика и её P-значение
F_test.statistic, F_test.pvalue

(198.58693315849226, 7.280959454662307e-39)

In [6]:
# уровень значимости
sign_level = 0.05
# Критическое значение F-распределения
f.ppf(q=1-sign_level, dfn=F_test.df_num, dfd=F_test.df_denom)

3.857969880147654

In [7]:
# степени свободы 
F_test.df_num, F_test.df_denom

(1.0, 565.0)

## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Отвергаем гипотезу $H_0$ (т.к. $F<F_{cr}$ или $P>\alpha$)

</div>

## F-тест (робастный)

Используем HC3-оценку ковариационной матрицы

In [8]:
# спецификация модели через формулу
mod = smf.ols(formula='np.log(output)~1+np.log(capital)+np.log(labour)+np.log(wage)', data=df)
# подгонка модели с робастной HC3-оценкой ковариационной матрицы
res_hc = mod.fit(cov_type='HC3')

In [9]:
F_test = res_hc.f_test('np.log(capital)+np.log(labour)+np.log(wage)=1')
print(F_test)

<F test: F=109.34016651228757, p=1.630280244154787e-23, df_denom=565, df_num=1>


In [10]:
# Тестовая статистика и её P-значение
F_test.statistic, F_test.pvalue

(109.34016651228757, 1.630280244154787e-23)

In [11]:
# уровень значимости
sign_level = 0.05
# Критическое значение F-распределения
f.ppf(q=1-sign_level, dfn=F_test.df_num, dfd=F_test.df_denom)

3.857969880147654

In [12]:
# альтернативно
W_test = res_hc.wald_test('np.log(capital)+np.log(labour)+np.log(wage)=1', use_f=True)
print(W_test)

<F test: F=array([[109.34016651]]), p=1.630280244154787e-23, df_denom=565, df_num=1>


## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Отвергаем гипотезу $H_0$ (т.к. $F<F_{cr}$ или $P>\alpha$)

</div>